<a href="https://colab.research.google.com/github/hyunjimoon/stan/blob/master/20Stan_golf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cmdstanpy
import pandas as pd
import numpy as np
from scipy.stats import beta
import matplotlib.pyplot as plt
import os
from cmdstanpy import CmdStanModel, cmdstan_path


def get_stan_model(code):
    code_hash = hashlib.sha1(code.encode('utf-8')).hexdigest()
    cache_path =  './cache/' + code_hash + '.pkl'
    try:
        with open(cache_path, 'rb') as file:
            model = pickle.load(file)
    except Exception:
        model = pystan.StanModel(model_code=code) ## c로 변환
        with open(cache_path, 'wb') as file:
            pickle.dump(model, file)
    return model

In [2]:
df = pd.read_csv('data/golf_data.txt')
new_df = pd.read_csv('data/golf_newdata.txt')

## Testing the fitted model on new data

Recently a local business school professor and golfer, Mark Broadie, came by my office with tons of new data. For simplicity we’ll just look here at the summary data, probabilities of the ball going into the hole for shots up to 75 feet from the hole. The graph below shows these new data (in red), along with our earlier dataset (in blue) and the already-fit geometry-based model from before, extending to the range of the new data.



![title](data/img/checking_alreadyfit.png)

Comparing the two datasets in the range 0-20 feet, the success rate is similar for longer putts but is much higher than before for the short putts. This could be a measurement issue, if the distances to the hole are only approximate for the old data, and it could also be that golfers are better than they used to be.

Beyond 20 feet, the empirical success rates become lower than would be predicted by the old model. These are much more difficult attempts, even after accounting for the increased angular precision required as distance goes up.

## A new model accounting for how hard the ball is hit
To get the ball in the hole, the angle isn’t the only thing you need to control; you also need to hit the ball just hard enough.

Mark Broadie added this to our model by introducing another parameter corresponding to the golfer’s control over distance. Supposing uu is the distance that golfer’s shot would travel if there were no hole, Broadie assumes that the putt will go in if (a) the angle allows the ball to go over the hole, and (b) uu is in the range [x,x+3]. That is the ball must be hit hard enough to reach the whole but not go too far. Factor (a) is what we have considered earlier; we must now add factor (b).

The following sketch, which is not to scale, illustrates the need for the distance as angle as well as the angle of the shot to be in some range, in this case the gray zone which represents the trajectories for which the ball would reach the hole and stay in it.

![title](data/img/golf_physical_model.png)

Broadie supposes that a golfer will aim to hit the ball one foot past the hole but with a multiplicative error in the shot’s potential distance, so that u=(x+1)⋅(1+error)u=(x+1)⋅(1+error), where the error has a normal distribution with mean 0 and standard deviation σdistanceσdistance. This new parameter σdistanceσdistance represents the uncertainty in the shot’s relative distance. In statistics notation, this model is,

$$ u∼normal(x+1,(x+1)\sigma_{distance }),$$
and the distance is acceptable if u∈[x,x+3], an event that has probability 
$$\Phi\left(\frac{2}{(x+1) \sigma_{distance}}\right)-\Phi\left(\frac{-1}{(x+1) \sigma_{distance}}\right)$$

Putting these together, the probability a shot goes in becomes, 

$$\left(2 \Phi\left(\frac{\sin ^{-1}((R-r)(x)}{\sigma_{\text {angle}}}\right)-1\right)\left(\Phi\left(\frac{2}{(x+1) \sigma_{\text {distance }}}\right)-\Phi\left(\frac{-1}{(x+1) \sigma_{\text {distance }}}\right)\right)$$

where we have renamed the parameter σσ from our earlier model to σangleσangle to distinguish it from the new σdistanceσdistance parameter. We write the new model in Stan, giving it the name `golf_angle_distance_2.stan` to convey that it is the second model in a series, and that it accounts both for angle and distance:

```
data {
  int J;
  int n[J];
  vector[J] x;
  int y[J];
  real r;
  real R;
  real overshot;
  real distance_tolerance;
}
transformed data {
  vector[J] threshold_angle = asin((R-r) ./ x);
}
parameters {
  real<lower=0> sigma_angle;
  real<lower=0> sigma_distance;
}
model {
  vector[J] p_angle = 2*Phi(threshold_angle / sigma_angle) - 1;
  vector[J] p_distance = Phi((distance_tolerance - overshot) ./ ((x + overshot)*sigma_distance)) -
               Phi((- overshot) ./ ((x + overshot)*sigma_distance));
  vector[J] p = p_angle .* p_distance;
  y ~ binomial(n, p);
  [sigma_angle, sigma_distance] ~ normal(0, 1);
}
generated quantities {
  real sigma_degrees = sigma_angle * 180 / pi();
}
```

Here we have defined `overshot` and `distance_tolerance` as data, which Broadie has specified as 1 and 3 feet, respectively. We might wonder why if the distance range is 3 feet, the overshot is not 1.5 feet. One reason could be that it is riskier to hit the ball too hard than too soft. In addition we assigned weakly informative half-normal(0,1) priors on the scale parameters, $\sigma_{angle}$ and $\sigma_{distance}$, which are required in this case to keep the computations stable.

In [3]:
golf_angle_distance_2_path = os.path.join('./stanfile','golf_angle_distance_2.stan')
golf_angle_distance_2_model = CmdStanModel(stan_file=golf_angle_distance_2_path)
golf_angle_distance_2_data = {
    "J": new_df.shape[0],
    "n": list(new_df.loc[:,'n']),
    "x": list(new_df.loc[:,'x']),
    "y": list(new_df.loc[:,'y']),
    "r": (1.68/2)/12,
    "R": (4.25/2)/12,
    "overshot": 1,
    "distance_tolerance": 3,
}
golf_angle_distance_2_fit = golf_angle_distance_2_model.sample(chains=5, cores=3, data=golf_angle_distance_2_data)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /Users/hyunjimoon/Dropbox/stan/stan/stanfile/golf_angle_distance_2
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:start chain 5
INFO:cmdstanpy:finish chain 5
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 3


## Fitting the new model to data
We fit the model to the new dataset.

![title](data/img/errormessage.png)

There is poor convergence, and we need to figure out what is going on here. (Problems with computation often indicate underlying problems with the model being fit. That’s the folk theorem of statistical computing.)

In [8]:
golf_angle_distance_2_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-365068.000000,171.989000,334.442000,-365894.000000,-364910.000000,-364909.000000,3.78129,0.288781,2.31330
sigma_angle,0.013868,0.000636,0.001193,0.013191,0.013307,0.016807,3.52151,0.268941,2.46142
sigma_distance,0.131105,0.006733,0.012703,0.099335,0.137084,0.138124,3.56017,0.271894,2.46444
sigma_degrees,0.794584,0.036424,0.068353,0.755758,0.762465,0.962978,3.52151,0.268941,2.46142


To understand what is happening, we graph the new data and the fitted model, accepting that this “fit,” based as it is on poorly-mixing chains, is only provisional:



![title](data/img/checking_modelfit.png)

To understand what is happening, we graph the new data and the fitted model, accepting that this “fit,” based as it is on poorly-mixing chains, is only provisional:

In [9]:
new_df.iloc[:5,]

,x,n,y
0,0.28,45198,45183
1,0.97,183020,182899
2,1.93,169503,168594
3,2.92,113094,108953
4,3.93,73855,64740


With such large values of $n_j$, the binomial likelihood enforces an extremely close fit at these first few points, and that drives the entire fit of the model.

To fix this problem we took the data model, $yj∼binomial(nj,pj)$, and added an independent error term to each observation. There is no easy way to add error directly to the binomial distribution—we could replace it with its overdispersed generalization, the beta-binomial, but this would not be appropriate here because the variance for each data point ii would still be roughly proportional to the sample size njnj, and our whole point here is to get away from this assumption and allow for model misspecification—so instead we first approximate the binomial data distribution by a normal and then add independent variance; thus:

$$
y_{j} / n_{j} \sim \text { normal }\left(p_{j}, \sqrt{p_{j}\left(1-p_{j}\right) / n_{j}+\sigma_{y}^{2}}\right)
$$

To write this in Stan there are some complications:

- y and n are integer variables, which we convert to vectors so that we can multiply and divide them.

- To perform componentwise multiplication or division using vectors, you need to use `.*` or `./` so that San knows not to try to perform vector/matrix multiplication and division. Stan is opposite from R in this way: Stan defaults to vector/matrix operations and has to be told otherwise, whereas R defaults to componentwise operations, and vector/matrix multiplication in R is indicated using the `%*%` operator.

We implement these via the following new code in the transformed data block:
```
vector[J] raw_proportions = to_vector(y) ./ to_vector(n);
```
And then in the model block:
```
  raw_proportions ~ normal(p, sqrt(p .* (1-p) ./ to_vector(n) + sigma_y^2));
```

To complete the model we add $\sigma_y$ to the parameters block and assign it a weakly informative half-normal(0,1) prior distribution. Here’s the new Stan program:
```
data {
  int J;
  int n[J];
  vector[J] x;
  int y[J];
  real r;
  real R;
  real overshot;
  real distance_tolerance;
}
transformed data {
  vector[J] threshold_angle = asin((R-r) ./ x);
  vector[J] raw_proportions = to_vector(y) ./ to_vector(n);
}
parameters {
  real<lower=0> sigma_angle;
  real<lower=0> sigma_distance;
  real<lower=0> sigma_y;
}
model {
  vector[J] p_angle = 2*Phi(threshold_angle / sigma_angle) - 1;
  vector[J] p_distance = Phi((distance_tolerance - overshot) ./ ((x + overshot)*sigma_distance)) -
               Phi((- overshot) ./ ((x + overshot)*sigma_distance));
  vector[J] p = p_angle .* p_distance;
  raw_proportions ~ normal(p, sqrt(p .* (1-p) ./ to_vector(n) + sigma_y^2));
  [sigma_angle, sigma_distance, sigma_y] ~ normal(0, 1);
}
generated quantities {
  real sigma_degrees = sigma_angle * 180 / pi();
}
```

We now fit this model to the data:

In [10]:
golf_angle_distance_3_path = os.path.join('./stanfile','golf_angle_distance_3.stan')
golf_angle_distance_3_model = CmdStanModel(stan_file=golf_angle_distance_3_path)
golf_angle_distance_3_data = {
    "J": new_df.shape[0],
    "n": list(new_df.loc[:,'n']),
    "x": list(new_df.loc[:,'x']),
    "y": list(new_df.loc[:,'y']),
    "r": (1.68/2)/12,
    "R": (4.25/2)/12,
    "overshot": 1,
    "distance_tolerance": 3,
}
golf_angle_distance_3_fit = golf_angle_distance_3_model.sample(chains=5, cores=3, data=golf_angle_distance_3_data)
golf_angle_distance_3_fit.summary()


INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /Users/hyunjimoon/Dropbox/stan/stan/stanfile/golf_angle_distance_3
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:start chain 5
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 5


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,146.580000,0.034004,1.316970,143.976000,146.948000,147.965000,1499.98,2195.08,1.00219
sigma_angle,0.017801,0.000002,0.000107,0.017628,0.017799,0.017972,1861.76,2724.52,1.00094
sigma_distance,0.080120,0.000031,0.001331,0.077941,0.080143,0.082251,1828.64,2676.05,1.00097
sigma_y,0.003067,0.000013,0.000604,0.002211,0.002996,0.004168,2233.74,3268.87,1.00061
sigma_degrees,1.019900,0.000142,0.006132,1.010040,1.019830,1.029720,1861.74,2724.49,1.00094


The new parameter estimates are:

- $\sigma_{angle}$ is estimated at 0.02, which when corresponds to $\sigma_{degrees}$  = 1.0. According to the fitted model, there is a standard deviation of 1.0 degree in the angles of putts taken by pro golfers. The estimate of $\sigma_{angle}$ has decreased compared to the earlier model that only had angular errors. This makes sense: now that distance errors have been included in the model, there is no need to explain so many of the missed shots using errors in angle.

- $\sigma_{distance}$  is estimated at 0.08. According to the fitted model, there is a standard deviation of 8% in the errors of distance.

- $\sigma_{y}$ is estimated at 0.003. The fitted model fits the aggregate data (success rate as a function of distance) to an accuracy of 0.3 percentage points.

And now we graph:

![title](data/img/checking_modelfit_2.png)

We can go further and plot the residuals from this fit. First we augment the Stan model to compute residuals in the generated quantities block.

Then we compute the posterior means of the residuals, $y_{j}/n_{j}−p_{j}$, then plot these vs. distance:

![title](data/img/residuals.png)

The residuals are small (see the scale of the yy-axis) and show no clear pattern, suggesting not that the model is perfect but that there are no clear ways to develop it further just given the current data.

## Problems with the model and potential improvements
The error term in the above model is a hack. It’s useful to allow the model not to fit the data perfectly, but it can’t be right to model these systematic errors as being independent. In this case, though, it doesn’t really matter, given how tiny these errors are: their estimated standard deviation is less than one percentage point.

The model has two parameters that were fixed as data: `distance_tolerance`, which was set to 3 (implying that the ball will only fall into the hole if it is hit on a trajectory that would go past the hole, but no more than 3 feet past) and `overshot`, which was set to 1 (implying that the golfer will aim 1 foot past the hole). In theory it would be possible to estimate either or both these parameters from the data. In practice, no way. The model already fits the data so well (as shown by the above graph) that there’s clearly no more information available to estimate any additional parameters. If we were to do so, the estimates would be highly noisy and unstable (if their prior is weak) or highly dependent on the prior (if an informative prior distribution is specified). Either way we don’t see the advantage of this sort of fit.

Just for laughs, though, we constructed such a model and fit it, just to see what would happen. We simply took our previous Stan program and moved these two parameters from the data block to the parameters block along with zero-boundary constraints:
```
real<lower=0> overshot;
real<lower=0> distance_tolerance;
```
And then in the model block we added weak priors centered at Broadie’s guesses and with wide uncertainties:

```  
overshot ~ normal(1, 5);
distance_tolerance ~ normal(3, 5);
  ```
Fitting this model to the data yields poor convergence and no real gain beyond the simpler version already fit in which overshot and distance_tolerance were set to fixed values.

The model is unrealistic in other ways, for example by assuming distance error is strictly proportional to distance aimed, and assuming independence of angular and distance errors. Presumably, angular error is higher for longer putts. Again, though, we can’t really investigate such things well using these data which are already such a good fit to the simple two-parameter model we have already fit.

Players vary in ability and golf courses vary in difficulty. Given more granular data, we should be able to fit a multilevel model allowing parameters to vary by player, golf course, and weather conditions.

# Summary of fitted models
We have two datasets, `golf` and `golf_new`, to which we fit several Stan models. First we fit `golf_logistic` and `golf_angle` to the `golf` dataset, then we fit `golf_angle` to the `golf_new` dataset and see a problem, then we fit `golf_angle_distance_2` and `golf_angle_distance_3` to `golf_new` and finally obtained a good fit, then we fit `golf_angle_distance_3_with_resids` which was the same model but also saving residuals. Finally, we fit `golf_angle_distance_4` to `golf_new` but we didn’t display the fit, we just discussed it.

# References
Don Berry (1995). Statistics: A Bayesian Perspective. Duxbury Press. The original golf dataset appears as an example in this book.

Mark Broadie (2018). Two simple putting models in golf. Linked from https://statmodeling.stat.columbia.edu/2019/03/21/new-golf-putting-data-and-a-new-golf-putting-model/. Here is the larger dataset and a document describing the model with angular and distance errors.

Andrew Gelman and Deborah Nolan (2002). A probability model for golf putting. Teaching Statistics 50, 151-153. Our first explanation of the angular-error model.

All code in this document is licensed under BSD 3-clause license and all text licensed under CC-BY-NC 4.0